# Eco-inspired Indicators

# Basecase (Linear Economy)

In [23]:
#importing packages
import numpy as np
import pandas as pd
import math
from scipy.special import logsumexp

#GitHub CSV file URL
basecase_csv_url = 'https://raw.githubusercontent.com/FarshidNazemi/ENA/main/data/Basecase%20(linear%20Economy).csv'
# Read CSV file, skip first two rows, and drop first two columns
df_basecase = pd.read_csv(basecase_csv_url, header=None, skiprows=2)
df_basecase = df_basecase.iloc[:, 2:]

# Building the total flow matrix
R = df_basecase.to_numpy()

F_R = 0 #refusal flow

#Internal Flow Matrix (F)

# Get the dimensions of the matrix
rows, cols = R.shape

# Specify how many rows and columns to remove
rows_to_remove = 4
cols_to_remove = 4

# Remove the last rows and columns
if rows_to_remove > 0 and cols_to_remove > 0:
    F = R[:rows - rows_to_remove, :cols - cols_to_remove]

#consumer demand (functional unit)
FU = F[1,2] #1 metric ton

#virgin input vector (Z)
# Get the fourth row from the end and store it in a vector
Z = R[-4]
Z = Z[:-4]

#Recovered input vector (P)
# Get the fourth row from the end and store it in a vector
P = R[-3]
P = P[:-4]

#Dissipation vector (Y)
# Get the fourth row from the end and store it in a vector
Y = R[:,-2]
Y = Y[:-4]

#Export vector (E)
# Get the fourth row from the end and store it in a vector
E = R[:,-1]
E = E[:-4]

#number of compartments
n=len(F)

#Total boundary input
TBI = np.sum(Z) + np.sum(P)

#Total Input (Output) Throughflows
T_i = np.sum(F, axis=0) + Z + P

#Flow Analysis (identifies non-dimentional flow intensities along indirect pathways)
epsilon = 1e-10
G = F/(T_i + epsilon)

#Dimentionless Integral intensities matrix
I=np.identity(n,dtype='float64')
N=np.linalg.inv(I-G)

#Development capacity
DC = 0.0

for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))
            
#Ascendency
ASC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            ASC += numerator * np.log2((numerator*np.sum(R) / denominator) + epsilon)

#Redundancy
PHI = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            PHI += -numerator * np.log2((numerator**2 / denominator) + epsilon)

#Development capacity
DC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))

#Ratio of ascendency to development capacity
R_ASC = ASC / DC
#Total System Throughflow
TST_flow = np.sum(F)

#Network Intensity
NI = TST_flow/FU

#Adjusted Network Intensity
NI_star = TST_flow/(FU+F_R)

#Robustness
ecological_resilience = -R_ASC*math.log(R_ASC)
Robustness = ecological_resilience/0.3678

#Mean Circularity Level (MCL)
m=8
Num=0.0
Denom=0.0
MTL=0.0
#Circularity Levels (CL): 1: Litter 2: Landfill 3: Energy Recovery/Compost 4: Open-loop recovery
#5: Refurbish/Recycle 6: Redistribute 7: Reuse 8: Refuse/Rethink
BLi=[0, #1: Litter
    T_i[4], #2: Landfill
     0, #3: Energy Recovery/Compost
     0, #4: Open-loop Recovery
     0, #5: Refursbish/Recycle
     0, #6: Redistribute
     0, #7: Reuse
     0 #8: Refuse/Rethink
    ]
CLi=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
for i in range (m):
    Num=Num+BLi[i]*CLi[i]
    Denom=Denom+BLi[i]
MCL=Num/Denom
MCL=(MCL-1)/(8-1) #Normalizing

#Loop Tightness
FU_c=(N[2,2]-1)/(N[2,2]+epsilon)*T_i[2]
LT=FU_c/FU

#Recovered-Renewable to Virgin ratio (R : V)
R_V = 0 #no recovery

#Resource Consumption Efficiency
RCE = FU/(np.sum(Z)+epsilon)

#Adjusted Resource Consumption Efficiency
RCE_star = (FU+F_R)/(np.sum(Z)+epsilon)

#EMF's Material Circularity Indicator (MCI)
V = 1.111    #Ratio of product composition from virgin material
W = 1.0555    #Waste (including production waste)
M = 1    #Mass of Product
MCI = 1 - (V+W)/(2*M)
if MCI<0:   #Normalizing MCI to a value between 0 and 1
    MCI=0
elif MCI>1:
        MCI=1

print("\033[1mEco-inspired Indicators:\033[0m")
print("Recovered-Renewable to Virgin ratio (R : V) =", R_V)
print("Resource Consumption efficiency (RCE) =", RCE)
print("Adjusted Resource Consumption efficiency (RCE_star) =", RCE_star)
print("Network Intensity (NI) =", NI)
print("Adjusted Network Intensity (NI_star) =", NI_star)
print("Robustness =", Robustness)
print ("Loop Tightness =", LT)
print ("Mean Circularity Level =", MCL)
print("----------------------------------------------------------")
print("Material Circularity Indicator (MCI) =", MCI)

Eco-inspired Indicators:
Recovered-Renewable to Virgin ratio (R : V) = 0
Resource Consumption efficiency (RCE) = 0.9000900089198839
Adjusted Resource Consumption efficiency (RCE_star) = 0.9000900089198839
Network Intensity (NI) = 4.0555
Adjusted Network Intensity (NI_star) = 4.0555
Robustness = 0.196043977287096
Loop Tightness = 0.0
Mean Circularity Level = 0.14285714285714285
----------------------------------------------------------
Material Circularity Indicator (MCI) = 0


# Reuse

In [24]:
#importing packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.special import logsumexp

#GitHub CSV file URL
basecase_csv_url = 'https://raw.githubusercontent.com/FarshidNazemi/ENA/main/data/Reuse.csv'
# Read CSV file, skip first two rows, and drop first two columns
df_basecase = pd.read_csv(basecase_csv_url, header=None, skiprows=2)
df_basecase = df_basecase.iloc[:, 2:]

# Building the total flow matrix
R = df_basecase.to_numpy()

F_R = 0 #refusal flow

#Internal Flow Matrix (F)

# Get the dimensions of the matrix
rows, cols = R.shape

# Specify how many rows and columns to remove
rows_to_remove = 4
cols_to_remove = 4

# Remove the last rows and columns
if rows_to_remove > 0 and cols_to_remove > 0:
    F = R[:rows - rows_to_remove, :cols - cols_to_remove]

#consumer demand (functional unit)
FU = F[1,2]+F[3,2] #1 metric ton

#virgin input vector (Z)
# Get the fourth row from the end and store it in a vector
Z = R[-4]
Z = Z[:-4]

#Recovered input vector (P)
# Get the fourth row from the end and store it in a vector
P = R[-3]
P = P[:-4]

#Dissipation vector (Y)
# Get the fourth row from the end and store it in a vector
Y = R[:,-2]
Y = Y[:-4]

#Export vector (E)
# Get the fourth row from the end and store it in a vector
E = R[:,-1]
E = E[:-4]

#number of compartments
n=len(F)

#Total boundary input
TBI = np.sum(Z) + np.sum(P)

#Total Input (Output) Throughflows
T_i = np.sum(F, axis=0) + Z + P

#Flow Analysis (identifies non-dimentional flow intensities along indirect pathways)
epsilon = 1e-10
G = F/(T_i + epsilon)

#Dimentionless Integral intensities matrix
I=np.identity(n,dtype='float64')
N=np.linalg.inv(I-G)

#Development capacity
DC = 0.0

for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))
            
#Ascendency
ASC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            ASC += numerator * np.log2((numerator*np.sum(R) / denominator) + epsilon)

#Redundancy
PHI = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            PHI += -numerator * np.log2((numerator**2 / denominator) + epsilon)

#Development capacity
DC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))

#Ratio of ascendency to development capacity
R_ASC = ASC / DC

#Total System Throughflow
TST_flow = np.sum(F)

#Network Intensity
NI = TST_flow/FU

#Adjusted Network Intensity
NI_star = TST_flow/(FU+F_R)

#Robustness
ecological_resilience = -R_ASC*math.log(R_ASC)
Robustness = ecological_resilience/0.3678

#Mean Circularity Level (MCL)
m=8
Num=0.0
Denom=0.0
MTL=0.0
#Circularity Levels (CL): 1: Litter 2: Landfill 3: Energy Recovery/Compost 4: Open-loop recovery
#5: Refurbish/Recycle 6: Redistribute 7: Reuse 8: Refuse/Rethink
BLi=[0, #1: Litter
    T_i[4], #2: Landfill
     0, #3: Energy Recovery/Compost
     0, #4: Open-loop Recovery
     0, #5: Refursbish/Recycle
     0, #6: Redistribute
     T_i[3], #7: Reuse
     0 #8: Refuse/Rethink
    ]
CLi=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
for i in range (m):
    Num=Num+BLi[i]*CLi[i]
    Denom=Denom+BLi[i]
MCL=Num/Denom
MCL=(MCL-1)/(8-1) #Normalizing

#Loop Tightness
FU_c=(N[2,2]-1)/(N[2,2]+epsilon)*T_i[2]
LT=FU_c/FU

#Recovered-Renewable to Virgin ratio (R : V)
R_V = F[3,2]/np.sum(Z) #no recovery

#Resource Consumption Efficiency
RCE = FU/(np.sum(Z)+epsilon)

#Adjusted Resource Consumption Efficiency
RCE_star = (FU+F_R)/(np.sum(Z)+epsilon)

#EMF's Material Circularity Indicator
V = 0.222    #Ratio of product composition from virgin material
W = 0.161    #Waste (excluding production waste)
M = 1    #Mass of Product
MCI = 1 - (V+W)/(2*M)
if MCI<0:   #Normalizing MCI to a value between 0 and 1
    MCI=0
elif MCI>1:
        MCI=1
        
print("\033[1mEco-inspired Indicators:\033[0m")
print("Recovered-Renewable to Virgin ratio (R : V) =", R_V)
print("Resource Consumption efficiency (RCE) =", RCE)
print("Adjusted Resource Consumption efficiency (RCE_star) =", RCE_star)
print("Network Intensity (NI) =", NI)
print("Adjusted Network Intensity (NI_star) =", NI_star)
print("Robustness =", Robustness)
print ("Loop Tightness =", LT)
print ("Mean Circularity Level =", MCL)
print("----------------------------------------------------------")
print("Material Circularity Indicator (MCI) =", MCI)

Eco-inspired Indicators:
Recovered-Renewable to Virgin ratio (R : V) = 3.6036036036036037
Resource Consumption efficiency (RCE) = 4.504504502475448
Adjusted Resource Consumption efficiency (RCE_star) = 4.504504502475448
Network Intensity (NI) = 2.3609999999999998
Adjusted Network Intensity (NI_star) = 2.3609999999999998
Robustness = 0.6557473807916919
Loop Tightness = 0.799999999824
Mean Circularity Level = 0.7580903162298511
----------------------------------------------------------
Material Circularity Indicator (MCI) = 0.8085


# Recycle

In [25]:
#importing packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.special import logsumexp

#GitHub CSV file URL
basecase_csv_url = 'https://raw.githubusercontent.com/FarshidNazemi/ENA/main/data/Recycle.csv'
# Read CSV file, skip first two rows, and drop first two columns
df_basecase = pd.read_csv(basecase_csv_url, header=None, skiprows=2)
df_basecase = df_basecase.iloc[:, 2:]

# Building the total flow matrix
R = df_basecase.to_numpy()

F_R = 0 #refusal flow

#Internal Flow Matrix (F)

# Get the dimensions of the matrix
rows, cols = R.shape

# Specify how many rows and columns to remove
rows_to_remove = 4
cols_to_remove = 4

# Remove the last rows and columns
if rows_to_remove > 0 and cols_to_remove > 0:
    F = R[:rows - rows_to_remove, :cols - cols_to_remove]

#consumer demand (functional unit)
FU = F[1,2] #1 metric ton

#virgin input vector (Z)
# Get the fourth row from the end and store it in a vector
Z = R[-4]
Z = Z[:-4]

#Recovered input vector (P)
# Get the fourth row from the end and store it in a vector
P = R[-3]
P = P[:-4]

#Dissipation vector (Y)
# Get the fourth row from the end and store it in a vector
Y = R[:,-2]
Y = Y[:-4]

#Export vector (E)
# Get the fourth row from the end and store it in a vector
E = R[:,-1]
E = E[:-4]

#number of compartments
n=len(F)

#Total boundary input
TBI = np.sum(Z) + np.sum(P)

#Total Input (Output) Throughflows
T_i = np.sum(F, axis=0) + Z + P

#Flow Analysis (identifies non-dimentional flow intensities along indirect pathways)
epsilon = 1e-10
G = F/(T_i + epsilon)

#Dimentionless Integral intensities matrix
I=np.identity(n,dtype='float64')
N=np.linalg.inv(I-G)

#Development capacity
DC = 0.0

for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))
            
#Ascendency
ASC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            ASC += numerator * np.log2((numerator*np.sum(R) / denominator) + epsilon)

#Redundancy
PHI = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            PHI += -numerator * np.log2((numerator**2 / denominator) + epsilon)

#Development capacity
DC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))

#Ratio of ascendency to development capacity
R_ASC = ASC / DC

#Total System Throughflow
TST_flow = np.sum(F)

#Network Intensity
NI = TST_flow/FU

#Adjusted Network Intensity
NI_star = TST_flow/(FU+F_R)

#Robustness
ecological_resilience = -R_ASC*math.log(R_ASC)
Robustness = ecological_resilience/0.3678

#Mean Circularity Level (MCL)
m=8
Num=0.0
Denom=0.0
MTL=0.0
#Circularity Levels (CL): 1: Litter 2: Landfill 3: Energy Recovery/Compost 4: Open-loop recovery
#5: Refurbish/Recycle 6: Redistribute 7: Reuse 8: Refuse/Rethink
BLi=[0, #1: Litter
    T_i[5], #2: Landfill
     0, #3: Energy Recovery/Compost
     0, #4: Open-loop Recovery
     T_i[4], #5: Refursbish/Recycle
     0, #6: Redistribute
     0, #7: Reuse
     0 #8: Refuse/Rethink
    ]
CLi=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
for i in range (m):
    Num=Num+BLi[i]*CLi[i]
    Denom=Denom+BLi[i]
MCL=Num/Denom
MCL=(MCL-1)/(8-1) #Normalizing

#Loop Tightness
FU_c=(N[2,2]-1)/(N[2,2]+epsilon)*T_i[2]
LT=FU_c/FU

#Recovered-Renewable to Virgin ratio (R : V)
R_V = F[4,0]/np.sum(Z) #no recovery

#Resource Consumption Efficiency
RCE = FU/(np.sum(Z)+epsilon)

#Adjusted Resource Consumption Efficiency
RCE_star = (FU+F_R)/(np.sum(Z)+epsilon)

#EMF's Material Circularity Indicator
V = 0.311    #Ratio of product composition from virgin material
W = 0.206    #Waste (excluding production waste)
M = 1    #Mass of Product
MCI = 1 - (V+W)/(2*M)
if MCI<0:   #Normalizing MCI to a value between 0 and 1
    MCI=0
elif MCI>1:
        MCI=1

print("\033[1mEco-inspired Indicators:\033[0m")
print("Recovered-Renewable to Virgin ratio (R : V) =", R_V)
print("Resource Consumption efficiency (RCE) =", RCE)
print("Adjusted Resource Consumption efficiency (RCE_star) =", RCE_star)
print("Network Intensity (NI) =", NI)
print("Adjusted Network Intensity (NI_star) =", NI_star)
print("Robustness =", Robustness)
print ("Loop Tightness =", LT)
print ("Mean Circularity Level =", MCL)
print("----------------------------------------------------------")
print("Material Circularity Indicator (MCI) =", MCI)

Eco-inspired Indicators:
Recovered-Renewable to Virgin ratio (R : V) = 2.572347266881029
Resource Consumption efficiency (RCE) = 3.2154340825673846
Adjusted Resource Consumption efficiency (RCE_star) = 3.2154340825673846
Network Intensity (NI) = 5.006
Adjusted Network Intensity (NI_star) = 5.006
Robustness = 0.4544968164858298
Loop Tightness = 0.7200720068277215
Mean Circularity Level = 0.4982231698649609
----------------------------------------------------------
Material Circularity Indicator (MCI) = 0.7415


# Downcycle

In [26]:
#importing packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.special import logsumexp

#GitHub CSV file URL
basecase_csv_url = 'https://raw.githubusercontent.com/FarshidNazemi/ENA/main/data/Downcycle.csv'
# Read CSV file, skip first two rows, and drop first two columns
df_basecase = pd.read_csv(basecase_csv_url, header=None, skiprows=2)
df_basecase = df_basecase.iloc[:, 2:]

# Building the total flow matrix
R = df_basecase.to_numpy()

F_R = 0 #refusal flow

#Internal Flow Matrix (F)

# Get the dimensions of the matrix
rows, cols = R.shape

# Specify how many rows and columns to remove
rows_to_remove = 4
cols_to_remove = 4

# Remove the last rows and columns
if rows_to_remove > 0 and cols_to_remove > 0:
    F = R[:rows - rows_to_remove, :cols - cols_to_remove]

#consumer demand (functional unit)
FU = F[1,2] #1 metric ton

#virgin input vector (Z)
# Get the fourth row from the end and store it in a vector
Z = R[-4]
Z = Z[:-4]

#Recovered input vector (P)
# Get the fourth row from the end and store it in a vector
P = R[-3]
P = P[:-4]

#Dissipation vector (Y)
# Get the fourth row from the end and store it in a vector
Y = R[:,-2]
Y = Y[:-4]

#Export vector (E)
# Get the fourth row from the end and store it in a vector
E = R[:,-1]
E = E[:-4]

#number of compartments
n=len(F)

#Total boundary input
TBI = np.sum(Z) + np.sum(P)

#Total Input (Output) Throughflows
T_i = np.sum(F, axis=0) + Z + P

#Flow Analysis (identifies non-dimentional flow intensities along indirect pathways)
epsilon = 1e-10
G = F/(T_i + epsilon)

#Dimentionless Integral intensities matrix
I=np.identity(n,dtype='float64')
N=np.linalg.inv(I-G)

#Development capacity
DC = 0.0

for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))
            
#Ascendency
ASC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            ASC += numerator * np.log2((numerator*np.sum(R) / denominator) + epsilon)

#Redundancy
PHI = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            PHI += -numerator * np.log2((numerator**2 / denominator) + epsilon)

#Development capacity
DC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))

#Ratio of ascendency to development capacity
R_ASC = ASC / DC

#Total System Throughflow
TST_flow = np.sum(F)

#Network Intensity
NI = TST_flow/FU

#Adjusted Network Intensity
NI_star = TST_flow/(FU+F_R)

#Robustness
ecological_resilience = -R_ASC*math.log(R_ASC)
Robustness = ecological_resilience/0.3678

#Mean Circularity Level (MCL)
m=8
Num=0.0
Denom=0.0
MTL=0.0
#Circularity Levels (CL): 1: Litter 2: Landfill 3: Energy Recovery/Compost 4: Open-loop recovery
#5: Refurbish/Recycle 6: Redistribute 7: Reuse 8: Refuse/Rethink
BLi=[0, #1: Litter
    T_i[5], #2: Landfill
     0, #3: Energy Recovery/Compost
     T_i[4], #4: Open-loop Recovery
     0, #5: Refursbish/Recycle
     0, #6: Redistribute
     0, #7: Reuse
     0 #8: Refuse/Rethink
    ]
CLi=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
for i in range (m):
    Num=Num+BLi[i]*CLi[i]
    Denom=Denom+BLi[i]
MCL=Num/Denom
MCL=(MCL-1)/(8-1) #Normalizing

#Loop Tightness
FU_c=(N[2,2]-1)/(N[2,2]+epsilon)*T_i[2]
LT=FU_c/FU

#Recovered-Renewable to Virgin ratio (R : V)
R_V = E[4]/np.sum(Z) #no recovery

#Resource Consumption Efficiency
RCE = FU/(np.sum(Z)+epsilon)

#Adjusted Resource Consumption Efficiency
RCE_star = (FU+F_R)/(np.sum(Z)+epsilon)

#EMF's Material Circularity Indicator
V = 1.111    #Ratio of product composition from virgin material
W = 0.206    #Waste (excluding production waste)
M = 1    #Mass of Product
MCI = 1 - (V+W)/(2*M)
if MCI<0:   #Normalizing MCI to a value between 0 and 1
    MCI=0
elif MCI>1:
        MCI=1

print("\033[1mEco-inspired Indicators:\033[0m")
print("Recovered-Renewable to Virgin ratio (R : V) =", R_V)
print("Resource Consumption efficiency (RCE) =", RCE)
print("Adjusted Resource Consumption efficiency (RCE_star) =", RCE_star)
print("Network Intensity (NI) =", NI)
print("Adjusted Network Intensity (NI_star) =", NI_star)
print("Robustness =", Robustness)
print ("Loop Tightness =", LT)
print ("Mean Circularity Level =", MCL)
print("----------------------------------------------------------")
print("Material Circularity Indicator (MCI) =", MCI)

Eco-inspired Indicators:
Recovered-Renewable to Virgin ratio (R : V) = 0.7200720072007201
Resource Consumption efficiency (RCE) = 0.9000900089198839
Adjusted Resource Consumption efficiency (RCE_star) = 0.9000900089198839
Network Intensity (NI) = 4.206
Adjusted Network Intensity (NI_star) = 4.206
Robustness = 0.2830346587166525
Loop Tightness = 0.0
Mean Circularity Level = 0.37976782752902155
----------------------------------------------------------
Material Circularity Indicator (MCI) = 0.3415


# Diverse EoL

In [28]:
#importing packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.special import logsumexp

#GitHub CSV file URL
basecase_csv_url = 'https://raw.githubusercontent.com/FarshidNazemi/ENA/main/data/Diverse%20EoL.csv'
# Read CSV file, skip first two rows, and drop first two columns
df_basecase = pd.read_csv(basecase_csv_url, header=None, skiprows=2)
df_basecase = df_basecase.iloc[:, 2:]

# Building the total flow matrix
R = df_basecase.to_numpy()

F_R = 0 #refusal flow

#Internal Flow Matrix (F)

# Get the dimensions of the matrix
rows, cols = R.shape

# Specify how many rows and columns to remove
rows_to_remove = 4
cols_to_remove = 4

# Remove the last rows and columns
if rows_to_remove > 0 and cols_to_remove > 0:
    F = R[:rows - rows_to_remove, :cols - cols_to_remove]

#consumer demand (functional unit)
FU = F[1,2] + F[4,2] #1 metric ton

#virgin input vector (Z)
# Get the fourth row from the end and store it in a vector
Z = R[-4]
Z = Z[:-4]

#Recovered input vector (P)
# Get the fourth row from the end and store it in a vector
P = R[-3]
P = P[:-4]

#Dissipation vector (Y)
# Get the fourth row from the end and store it in a vector
Y = R[:,-2]
Y = Y[:-4]

#Export vector (E)
# Get the fourth row from the end and store it in a vector
E = R[:,-1]
E = E[:-4]

#number of compartments
n=len(F)

#Total boundary input
TBI = np.sum(Z) + np.sum(P)

#Total Input (Output) Throughflows
T_i = np.sum(F, axis=0) + Z + P

#Flow Analysis (identifies non-dimentional flow intensities along indirect pathways)
epsilon = 1e-10
G = F/(T_i + epsilon)

#Dimentionless Integral intensities matrix
I=np.identity(n,dtype='float64')
N=np.linalg.inv(I-G)

#Development capacity
DC = 0.0

for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))
            
#Ascendency
ASC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            ASC += numerator * np.log2((numerator*np.sum(R) / denominator) + epsilon)

#Redundancy
PHI = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            PHI += -numerator * np.log2((numerator**2 / denominator) + epsilon)

#Development capacity
DC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))

#Ratio of ascendency to development capacity
R_ASC = ASC / DC

#Total System Throughflow
TST_flow = np.sum(F)

#Network Intensity
NI = TST_flow/FU

#Adjusted Network Intensity
NI_star = TST_flow/(FU+F_R)

#Robustness
ecological_resilience = -R_ASC*math.log(R_ASC)
Robustness = ecological_resilience/0.3678

#Mean Circularity Level (MCL)
m=8
Num=0.0
Denom=0.0
MTL=0.0
#Circularity Levels (CL): 1: Litter 2: Landfill 3: Energy Recovery/Compost 4: Open-loop recovery
#5: Refurbish/Recycle 6: Redistribute 7: Reuse 8: Refuse/Rethink
BLi=[0, #1: Litter
    T_i[7], #2: Landfill
     0, #3: Energy Recovery/Compost
     T_i[6], #4: Open-loop Recovery
     T_i[5], #5: Refursbish/Recycle
     0, #6: Redistribute
     T_i[4], #7: Reuse
     0 #8: Refuse/Rethink
    ]
CLi=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
for i in range (m):
    Num=Num+BLi[i]*CLi[i]
    Denom=Denom+BLi[i]
MCL=Num/Denom
MCL=(MCL-1)/(8-1) #Normalizing

#Loop Tightness
FU_c=(N[2,2]-1)/(N[2,2]+epsilon)*T_i[2]
LT=FU_c/FU

#Recovered-Renewable to Virgin ratio (R : V)
R_V = (E[6]+F[4,2]+F[5,0])/np.sum(Z) #no recovery

#Resource Consumption Efficiency
RCE = FU/(np.sum(Z)+epsilon)

#Adjusted Resource Consumption Efficiency
RCE_star = (FU+F_R)/(np.sum(Z)+epsilon)

#EMF's Material Circularity Indicator
V = 0.466    #Ratio of product composition from virgin material
W = 0.183    #Waste (excluding production waste)
M = 1    #Mass of Product
MCI = 1 - (V+W)/(2*M)
if MCI<0:   #Normalizing MCI to a value between 0 and 1
    MCI=0
elif MCI>1:
        MCI=1

print("\033[1mEco-inspired Indicators:\033[0m")
print("Recovered-Renewable to Virgin ratio (R : V) =", R_V)
print("Resource Consumption efficiency (RCE) =", RCE)
print("Adjusted Resource Consumption efficiency (RCE_star) =", RCE_star)
print("Network Intensity (NI) =", NI)
print("Adjusted Network Intensity (NI_star) =", NI_star)
print("Robustness =", Robustness)
print ("Loop Tightness =", LT)
print ("Mean Circularity Level =", MCL)
print("----------------------------------------------------------")
print("Material Circularity Indicator (MCI) =", MCI)

Eco-inspired Indicators:
Recovered-Renewable to Virgin ratio (R : V) = 1.7167381974248928
Resource Consumption efficiency (RCE) = 2.1459227463206174
Adjusted Resource Consumption efficiency (RCE_star) = 2.1459227463206174
Network Intensity (NI) = 3.6830000000000003
Adjusted Network Intensity (NI_star) = 3.6830000000000003
Robustness = 0.740273301352754
Loop Tightness = 0.5801801798526128
Mean Circularity Level = 0.5921908893709328
----------------------------------------------------------
Material Circularity Indicator (MCI) = 0.6755


# Refuse

In [29]:
#importing packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.special import logsumexp

#GitHub CSV file URL
basecase_csv_url = 'https://raw.githubusercontent.com/FarshidNazemi/ENA/main/data/Refuse.csv'
# Read CSV file, skip first two rows, and drop first two columns
df_basecase = pd.read_csv(basecase_csv_url, header=None, skiprows=2)
df_basecase = df_basecase.iloc[:, 2:]

# Building the total flow matrix
R = df_basecase.to_numpy()

F_R = 0.8 #refusal flow

#Internal Flow Matrix (F)

# Get the dimensions of the matrix
rows, cols = R.shape

# Specify how many rows and columns to remove
rows_to_remove = 4
cols_to_remove = 4

# Remove the last rows and columns
if rows_to_remove > 0 and cols_to_remove > 0:
    F = R[:rows - rows_to_remove, :cols - cols_to_remove]

#consumer demand (functional unit) after refusal
FU = F[1,2] #0.2 metric ton

#initial consumer demand before refusal
D_init = 1 #1 metric ton

#virgin input vector (Z)
# Get the fourth row from the end and store it in a vector
Z = R[-4]
Z = Z[:-4]

#Recovered input vector (P)
# Get the fourth row from the end and store it in a vector
P = R[-3]
P = P[:-4]

#Dissipation vector (Y)
# Get the fourth row from the end and store it in a vector
Y = R[:,-2]
Y = Y[:-4]

#Export vector (E)
# Get the fourth row from the end and store it in a vector
E = R[:,-1]
E = E[:-4]

#number of compartments
n=len(F)

#Total boundary input
TBI = np.sum(Z) + np.sum(P)

#Total Input (Output) Throughflows
T_i = np.sum(F, axis=0) + Z + P

#Flow Analysis (identifies non-dimentional flow intensities along indirect pathways)
epsilon = 1e-10
G = F/(T_i + epsilon)

#Dimentionless Integral intensities matrix
I=np.identity(n,dtype='float64')
N=np.linalg.inv(I-G)

#Development capacity
DC = 0.0

for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))
            
#Ascendency
ASC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            ASC += numerator * np.log2((numerator*np.sum(R) / denominator) + epsilon)

#Redundancy
PHI = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            PHI += -numerator * np.log2((numerator**2 / denominator) + epsilon)

#Development capacity
DC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))

#Ratio of ascendency to development capacity
R_ASC = ASC / DC

#Total System Throughflow
TST_flow = np.sum(F)

#Network Intensity
NI = TST_flow/FU

#Adjusted Network Intensity
NI_star = TST_flow/(FU+F_R)

#Adjusted Network Intensity
NI_star = TST_flow/(FU+F_R)

#Robustness
ecological_resilience = -R_ASC*math.log(R_ASC)
Robustness = ecological_resilience/0.3678

#Mean Circularity Level (MCL)
m=8
Num=0.0
Denom=0.0
MTL=0.0
#Circularity Levels (CL): 1: Litter 2: Landfill 3: Energy Recovery/Compost 4: Open-loop recovery
#5: Refurbish/Recycle 6: Redistribute 7: Reuse 8: Refuse/Rethink
BLi=[0, #1: Litter
    T_i[4], #2: Landfill
     0, #3: Energy Recovery/Compost
     0, #4: Open-loop Recovery
     0, #5: Refursbish/Recycle
     0, #6: Redistribute
     0, #7: Reuse
     D_init-FU #8: Refuse/Rethink
    ]
CLi=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
for i in range (m):
    Num=Num+BLi[i]*CLi[i]
    Denom=Denom+BLi[i]
MCL=Num/Denom
MCL=(MCL-1)/(8-1) #Normalizing

#Loop Tightness
FU_c=(N[2,2]-1)/(N[2,2]+epsilon)*T_i[2]
LT=FU_c/FU

#Recovered-Renewable to Virgin ratio (R : V)
R_V = 0 #no recovery


#Resource Consumption Efficiency
RCE = FU/(np.sum(Z)+epsilon)


#Adjusted Resource Consumption Efficiency
RCE_star = (FU+F_R)/(np.sum(Z)+epsilon)


#EMF's Material Circularity Indicator
V = 0.222    #Ratio of product composition from virgin material
W = 0.211    #Waste (excluding production waste)
M = 0.2    #Mass of Product
MCI = 1 - (V+W)/(2*M)
if MCI<0:   #Normalizing MCI to a value between 0 and 1
    MCI=0
elif MCI>1:
        MCI=1

print("\033[1mEco-inspired Indicators:\033[0m")
print("Recovered-Renewable to Virgin ratio (R : V) =", R_V)
print("Resource Consumption efficiency (RCE) =", RCE)
print("Adjusted Resource Consumption efficiency (RCE_star) =", RCE_star)
print("Network Intensity (NI) =", NI)
print("Adjusted Network Intensity (NI_star) =", NI_star)
print("Robustness =", Robustness)
print ("Loop Tightness =", LT)
print ("Mean Circularity Level =", MCL)
print("----------------------------------------------------------")
print("Material Circularity Indicator (MCI) =", MCI)

Eco-inspired Indicators:
Recovered-Renewable to Virgin ratio (R : V) = 0
Resource Consumption efficiency (RCE) = 0.9009009004950896
Adjusted Resource Consumption efficiency (RCE_star) = 4.504504502475448
Network Intensity (NI) = 4.055
Adjusted Network Intensity (NI_star) = 0.811
Robustness = 0.19487307355830358
Loop Tightness = 0.0
Mean Circularity Level = 0.821110640101738
----------------------------------------------------------
Material Circularity Indicator (MCI) = 0


# Best Case: Refuse + Diverse EoL 

In [32]:
#importing packages
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy.special import logsumexp

#GitHub CSV file URL
basecase_csv_url = 'https://raw.githubusercontent.com/FarshidNazemi/ENA/main/data/Best%20Case%20(Refuse%2BDiverse%20EoL).csv'
# Read CSV file, skip first two rows, and drop first two columns
df_basecase = pd.read_csv(basecase_csv_url, header=None, skiprows=2)
df_basecase = df_basecase.iloc[:, 2:]

# Building the total flow matrix
R = df_basecase.to_numpy()
F_R = 0.8 #refusal flow

#Internal Flow Matrix (F)

# Get the dimensions of the matrix
rows, cols = R.shape

# Specify how many rows and columns to remove
rows_to_remove = 4
cols_to_remove = 4

# Remove the last rows and columns
if rows_to_remove > 0 and cols_to_remove > 0:
    F = R[:rows - rows_to_remove, :cols - cols_to_remove]

#consumer demand (functional unit) after refusal
FU = F[1,2] + F[4,2] #0.2 metric ton

#initial consumer demand before refusal
D_init = 1 #1 metric ton

#virgin input vector (Z)
# Get the fourth row from the end and store it in a vector
Z = R[-4]
Z = Z[:-4]

#Recovered input vector (P)
# Get the fourth row from the end and store it in a vector
P = R[-3]
P = P[:-4]

#Dissipation vector (Y)
# Get the fourth row from the end and store it in a vector
Y = R[:,-2]
Y = Y[:-4]

#Export vector (E)
# Get the fourth row from the end and store it in a vector
E = R[:,-1]
E = E[:-4]

#number of compartments
n=len(F)

#Total boundary input
TBI = np.sum(Z) + np.sum(P)

#Total Input (Output) Throughflows
T_i = np.sum(F, axis=0) + Z + P

#Flow Analysis (identifies non-dimentional flow intensities along indirect pathways)
epsilon = 1e-10
G = F/(T_i + epsilon)

#Dimentionless Integral intensities matrix
I=np.identity(n,dtype='float64')
N=np.linalg.inv(I-G)

#Development capacity
DC = 0.0

for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))
            
#Ascendency
ASC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            ASC += numerator * np.log2((numerator*np.sum(R) / denominator) + epsilon)

#Redundancy
PHI = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            numerator = R[i, j]
            denominator = np.sum(R[i, :]+ epsilon) * (np.sum(R[:, j]) + epsilon)
            PHI += -numerator * np.log2((numerator**2 / denominator) + epsilon)

#Development capacity
DC = 0.0
for i in range(len(R)):
    for j in range(len(R)):
        if i != j:
            DC += -(R[i, j] * np.log2(R[i, j] / np.sum(R) + epsilon))

#Ratio of ascendency to development capacity
R_ASC = ASC / DC

#Total System Throughflow
TST_flow = np.sum(F)

#Network Intensity
NI = TST_flow/FU

#Adjusted Network Intensity
NI_star = TST_flow/(FU+F_R)

#Robustness
ecological_resilience = -R_ASC*math.log(R_ASC)
Robustness = ecological_resilience/0.3678

#Mean Circularity Level (MCL)
m=8
Num=0.0
Denom=0.0
MTL=0.0
#Circularity Levels (CL): 1: Litter 2: Landfill 3: Energy Recovery/Compost 4: Open-loop recovery
#5: Refurbish/Recycle 6: Redistribute 7: Reuse 8: Refuse/Rethink
BLi=[0, #1: Litter
    T_i[7], #2: Landfill
     0, #3: Energy Recovery/Compost
     T_i[6], #4: Open-loop Recovery
     T_i[5], #5: Refursbish/Recycle
     0, #6: Redistribute
     T_i[4], #7: Reuse
     F_R #8: Refuse/Rethink
    ]
CLi=[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0]
for i in range (m):
    Num=Num+BLi[i]*CLi[i]
    Denom=Denom+BLi[i]
MCL=Num/Denom
MCL=(MCL-1)/(8-1) #Normalizing

#Loop Tightness
FU_c=(N[2,2]-1)/(N[2,2]+epsilon)*T_i[2]
LT=FU_c/FU

#Recovered-Renewable to Virgin ratio (R : V)
R_V = (E[6]+F[4,2]+F[5,0])/np.sum(Z) #no recovery

#Resource Consumption Efficiency
RCE = FU/(np.sum(Z)+epsilon)

#Adjusted Resource Consumption Efficiency
RCE_star = (FU+F_R)/(np.sum(Z)+epsilon)

#EMF's Material Circularity Indicator (MCI)
V = 0.093   #Ratio of product composition from virgin material
W = 0.039       #Waste (excluding production waste)
M = 0.2    #Mass of Product
MCI = 1 - (V+W)/(2*M)



Eco-inspired Indicators:
Recovered-Renewable to Virgin ratio (R : V) = 1.7204301075268817
Resource Consumption efficiency (RCE) = 2.150537632096196
Adjusted Resource Consumption efficiency (RCE_star) = 10.75268816048098
Network Intensity (NI) = 3.4835000000000003
Adjusted Network Intensity (NI_star) = 0.6967000000000001
Robustness = 0.7059529641173917
Loop Tightness = 0.5804511262775637
Mean Circularity Level = 0.9076465157298571
----------------------------------------------------------
Material Circularity Indicator (MCI) = 0.6699999999999999
